In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import random

In [3]:
# Load CSV (update the path if needed)
df = pd.read_csv(r'C:\Users\Admin\Desktop\Github Projects\Text Generation\data\dataset.csv', on_bad_lines='skip')

# Extract text column, drop missing values, join into one big string
text = " ".join(df['text'].dropna().astype(str)).lower()

print(f'Total characters in text: {len(text)}')


Total characters in text: 35695884


In [4]:
# Create vocabulary of unique characters
vocab = sorted(set(text))
print(f'Vocabulary size: {len(vocab)}')

# Character to index mapping and reverse
char2idx = {c: i for i, c in enumerate(vocab)}
idx2char = np.array(vocab)

# Convert entire text to integers
text_as_int = np.array([char2idx[c] for c in text])


Vocabulary size: 104


In [5]:
seq_length = 100  # input length per example

# Create dataset from integer encoded text
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# Batch into sequences (input + target)
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

# Split sequences into input and target
def split_input_target(chunk):
    return chunk[:-1], chunk[1:]

dataset = sequences.map(split_input_target)

# Shuffle and batch dataset
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


In [6]:
vocab_size = len(vocab)
embedding_dim = 64
rnn_units = 128

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_shape=(None,)),
    tf.keras.layers.LSTM(rnn_units, return_sequences=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
])


def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

model.summary()


c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, None, 64)       │         6,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, None, 128)      │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, None, 104)      │        13,416 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 118,888 (464.41 KB)

 Trainable params: 118,888 (464.41 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
EPOCHS = 20
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/20
5522/5522 ━━━━━━━━━━━━━━━━━━━━ 365s 66ms/step - loss: 2.1014
Epoch 2/20
5522/5522 ━━━━━━━━━━━━━━━━━━━━ 400s 72ms/step - loss: 1.4841
Epoch 3/20
5522/5522 ━━━━━━━━━━━━━━━━━━━━ 404s 73ms/step - loss: 1.3967
Epoch 4/20
5522/5522 ━━━━━━━━━━━━━━━━━━━━ 378s 68ms/step - loss: 1.3604
Epoch 5/20
5522/5522 ━━━━━━━━━━━━━━━━━━━━ 386s 70ms/step - loss: 1.3391
Epoch 6/20
5522/5522 ━━━━━━━━━━━━━━━━━━━━ 388s 70ms/step - loss: 1.3246
Epoch 7/20
5522/5522 ━━━━━━━━━━━━━━━━━━━━ 390s 71ms/step - loss: 1.3143
Epoch 8/20
5522/5522 ━━━━━━━━━━━━━━━━━━━━ 390s 70ms/step - loss: 1.3063
Epoch 9/20
5522/5522 ━━━━━━━━━━━━━━━━━━━━ 395s 71ms/step - loss: 1.2998
Epoch 10/20
5522/5522 ━━━━━━━━━━━━━━━━━━━━ 405s 73ms/step - loss: 1.2945
Epoch 11/20
5522/5522 ━━━━━━━━━━━━━━━━━━━━ 416s 75ms/step - loss: 1.2901
Epoch 12/20
5522/5522 ━━━━━━━━━━━━━━━━━━━━ 400s 72ms/step - loss: 1.2866
Epoch 13/20
5522/5522 ━━━━━━━━━━━━━━━━━━━━ 406s 73ms/step - loss: 1.2833
Epoch 14/20
5522/5522 ━━━━━━━━━━━━━━━━━━━━ 411s 74ms/step - 

In [8]:
def generate_text(model, start_string, num_generate=100, temperature=1.0):
    input_eval = [char2idx.get(s, 0) for s in start_string.lower()]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    model.layers[1].reset_states()  # Reset LSTM layer states

    for _ in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0) / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)


In [9]:
print(generate_text(model, start_string="The ", num_generate=200, temperature=0.8))

The pply cos and erecid or, s the caictheredallluts d pre a th n th vounge s wilas fas co ame it an son’sakiofumengild a frg lin tenthay omis pon cilige leiacespp col athan ticeconas tise n dins s ingowhe
